In [1]:
## Population response

In [ ]:
%matplotlib inline
from brian2 import *

avgPot = []
## Number of input and output neurons
N = 10
F = 2 * Hz

eqs = '''
dpreVolt/dt=r : 1
r : Hz
'''
poissonNeurons = PoissonGroup(N, rates=F)

#store()
#input_spikes = SpikeMonitor(poissonNeurons)
#run(1000 * ms)
#plot(input_spikes.t, input_spikes.i, '.k')
#xlabel('Time (ms)')
#ylabel('Neuron index')
#show()
#restore()

In [ ]:
## Let us initialize variables for the output neurons.
numOut = 1

tau_e = 3 * ms # Decay current
tau_m = 10 * ms # Decay of membrane potential
Rm = 100 * Mohm # Membrane resistance

eqs_neuron = '''
dv/dt=(Rm*I-v)/tau_m:volt
dI/dt=-I/tau_e:amp
'''
outputNeurons = NeuronGroup(numOut, model=eqs_neuron)

In [ ]:
## Let's put synapses between these groups of neurons.

A_SE = 250 * pA # Absolute synaptic efficacy

taud=100*ms # Decay time synaptic depression
tauf=3*ms # Decay time synaptic facilitation
U=.1 # Release probability

In [ ]:
S=Synapses(poissonNeurons, outputNeurons,
           model='''w : amp
                    dx/dt=(1-x)/taud : 1 (clock-driven)
                    du/dt=(-u)/tauf : 1 (clock-driven)
                 ''',
           on_pre=''' u+=U*(1-u)
                  I+=w*u*x
                  x-= x*u''')

                  
S.connect( 'j==0') # all to one connection
S.w=A_SE # Initializing synaptic weights to absolute synaptic efficacy

# Initialization of STP variables

S.x = 1 # Fraction of resources that remain available after neurotransmitter depletion 
S.u = U # Fraction of available resources ready for use (release probability)

In [ ]:
## Setting up monitors
trace = StateMonitor(outputNeurons, 'v', record=[0])
u_vals = StateMonitor(S, 'u', record=[0])
x_vals = StateMonitor(S, 'x', record=[0])

#monitors = [trace, u_vals, x_vals]
#net = Network(collect())  # automatically include G and S
#net.add(monitors)  # manually add the monitors

run(1000 * ms)

plot(trace.t/ms, trace.v[0] / mV)
avgPot.append(np.average(trace.v[0]))
title('Vm_' + str(numOut))
tight_layout()
show()

In [ ]:
plot(trace.t/ms, u_vals.u[0])
plot(trace.t/ms, x_vals.x[0])
title('Vm_' + str(numOut))
tight_layout()
show()